In [ ]:
from google.colab import drive
drive.mount('/content/gdrive',force_remount=True)
!ls -lt '/content/gdrive/My Drive/' 
import numpy as np
import tensorflow as tf
from tensorflow.keras.layers import Dense, Flatten, MaxPooling1D
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Conv1D, Dropout
from tensorflow.keras.optimizers import Adam, RMSprop
from tensorflow.keras.losses import BinaryCrossentropy
from tensorflow.keras.metrics import binary_accuracy, AUC
from sklearn.utils import shuffle

In [ ]:
def model_creation():
  n_timesteps, n_features = 16384, 1
  input_shape=(n_timesteps,n_features)
  model=Sequential()

  model.add(Conv1D(filters=16, kernel_size=16, activation='relu', input_shape=input_shape))
  model.add(MaxPooling1D(pool_size=8))

  model.add(Conv1D(32, kernel_size=8, activation='relu'))
  model.add(MaxPooling1D(pool_size=8))

  model.add(Conv1D(64, kernel_size=8, activation='relu'))
  #model.add(Conv1D(16, kernel_size=16, activation='relu'))
  model.add(MaxPooling1D(pool_size=4))

  model.add(Conv1D(128, kernel_size=8, activation='relu'))
  #model.add(Conv1D(32, kernel_size=16, activation='relu'))
  model.add(MaxPooling1D(pool_size=4))

  model.add(Flatten())

  model.add(Dense(128, activation='relu'))
  model.add(Dropout(0.3))

  model.add(Dense(64, activation='relu'))
  model.add(Dropout(0.3))

  model.add(Dense(1, activation='sigmoid'))

  model.compile(optimizer= Adam(learning_rate=0.002, beta_1=0.9, beta_2=0.999, epsilon=1e-08),
                loss='binary_crossentropy',
                metrics=['binary_accuracy', 'accuracy'])
  model.summary()
  return model

In [ ]:
model = model_creation()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 conv1d (Conv1D)             (None, 16369, 16)         272       
                                                                 
 max_pooling1d (MaxPooling1D  (None, 2046, 16)         0         
 )                                                               
                                                                 
 conv1d_1 (Conv1D)           (None, 2039, 32)          4128      
                                                                 
 max_pooling1d_1 (MaxPooling  (None, 254, 32)          0         
 1D)                                                             
                                                                 
 conv1d_2 (Conv1D)           (None, 247, 64)           16448     
                                                                 
 max_pooling1d_2 (MaxPooling  (None, 61, 64)           0

In [ ]:
# training file locations
postrainfoldername = '/content/gdrive/MyDrive/Positive_Training_DATA'
negtrainfoldername = '/content/gdrive/MyDrive/Negative_Train_DATA'

# postive and negative prefixes
postrainprefix = 111100000
negtrainprefix = 101100000

#samples of pos and neg training data
postrainsamples = 24576
negtrainsamples = 8192

# initialises global variables 
def initialise():
  global postrainprefix
  global negtrainprefix
  postrainprefix = 111100000
  negtrainprefix = 101100000

# number of batches
batch_looping_times = 64

# number of samples in each batch
samples = (postrainsamples//batch_looping_times) +(negtrainsamples//batch_looping_times)

print(samples)
y=np.concatenate((np.ones(((postrainsamples//batch_looping_times), 1)), np.zeros(((negtrainsamples//batch_looping_times), 1))))
print(y.shape)

512
(512, 1)


In [ ]:
def batchload_train():
  global y
  global postrainprefix
  global negtrainprefix
  # print(samples)
  X=np.zeros((samples,8192*2))
  # y=np.zeros((samples,1))
  for i in range(postrainsamples//batch_looping_times):
    posfile_name = postrainfoldername+'/'+str(postrainprefix+i+1)+'.npy'
    X[i] = np.load(posfile_name)
    # y[i] = 1
  for i in range(negtrainsamples//batch_looping_times):
    negfile_name = negtrainfoldername+'/'+str(negtrainprefix+i+1)+'.npy'
    X[i+(postrainsamples//batch_looping_times)] = np.load(negfile_name)
    # y[i] = 0

  X = X.reshape(samples, 16384,1)
  
  return X, y

# trainX, trainy = batchload_train()


In [ ]:
def get_data():
  global postrainprefix
  global negtrainprefix
  global y
  initialise()
  while True:

    trainX, trainy = batchload_train()
    # train_dataset = tf.data.Dataset.from_tensor_slices((trainX, trainy)).shuffle(729)
    # train_dataset = train_dataset.batch(batch_size)
    trainX, trainy = shuffle(trainX, trainy, random_state=53)
    postrainprefix += (postrainsamples//batch_looping_times)
    negtrainprefix += (negtrainsamples//batch_looping_times)
    if (postrainprefix >= 111124575):
      initialise()
    yield trainX, trainy



In [ ]:
# training file locations
posvalfoldername = '/content/gdrive/MyDrive/Positive_Validation_DATA'
negvalfoldername = '/content/gdrive/MyDrive/Negative_Validation_DATA'

# postive and negative prefixes
posvalprefix = 110100000
negvalprefix = 100100000

#samples of pos and neg training data
posvalsamples = 3072
negvalsamples = 1024

# initialises global variables 
def initialise_val():
  global posvalprefix
  global negvalprefix
  posvalprefix = 110100000
  negvalprefix = 100100000

# number of batches
# batch_looping_times = 64

# number of samples in each batch
val_samples=  (posvalsamples//batch_looping_times) +(negvalsamples//batch_looping_times)

print(val_samples)
val_y=np.concatenate((np.ones(((posvalsamples//batch_looping_times), 1)), np.zeros(((negvalsamples//batch_looping_times), 1))))
print(val_y.shape)

64
(64, 1)


In [ ]:
def batchload_val():
  global val_y
  global val_samples
  # print(samples)
  X=np.zeros((val_samples,8192*2))
  # y=np.zeros((samples,1))
  for i in range(posvalsamples//batch_looping_times):
    posfile_name = posvalfoldername+'/'+str(posvalprefix+i+1)+'.npy'
    X[i] = np.load(posfile_name)
    # y[i] = 1
  for i in range(negvalsamples//batch_looping_times):
    negfile_name = negvalfoldername+'/'+str(negvalprefix+i+1)+'.npy'
    X[i+(posvalsamples//batch_looping_times)] = np.load(negfile_name)
    # y[i] = 0

  X = X.reshape(val_samples, 16384,1)
  
  return X, val_y


In [ ]:
def get_val_data():
  global posvalprefix
  global negvalprefix
  global val_y
  initialise_val()
  while True:
    val_X, val_y = batchload_val()
    # train_dataset = tf.data.Dataset.from_tensor_slices((trainX, trainy)).shuffle(729)
    # train_dataset = train_dataset.batch(batch_size)
    val_X, val_y = shuffle(val_X, val_y, random_state=10)
    posvalprefix += (posvalsamples//batch_looping_times)
    negvalprefix += (negvalsamples//batch_looping_times)
    if (posvalprefix >= 110103071):
      initialise_val()
    yield val_X, val_y

In [ ]:
from tensorflow.keras.callbacks import ModelCheckpoint
checkpoint_path = "/content/gdrive/My Drive/BBH_Classification_3.ckpt"


# Create a callback that saves the model's weights every 5 epochs
cp_callback = ModelCheckpoint(
    filepath=checkpoint_path, 
    verbose=0, 
    save_weights_only=True,
    save_freq = 1)

model.load_weights("/content/gdrive/My Drive/BBH_Classification_2.ckpt")

In [ ]:
from keras.models import load_model
def model_training():

    model.fit(get_data(), epochs=5, steps_per_epoch=64, verbose = 1, shuffle=
                        True, callbacks=[cp_callback], batch_size = 512, use_multiprocessing = True)
    model.save("/content/gdrive/MyDrive/BBH_Classification_Model_3.h5")

In [ ]:
model_training()

Epoch 1/5
64/64 [==============================] - 4679s 74s/step - loss: 0.0780 - binary_accuracy: 0.9743 - accuracy: 0.9743
Epoch 2/5
64/64 [==============================] - 393s 6s/step - loss: 0.0746 - binary_accuracy: 0.9763 - accuracy: 0.9763
Epoch 3/5
64/64 [==============================] - 377s 6s/step - loss: 0.0688 - binary_accuracy: 0.9777 - accuracy: 0.9777
Epoch 4/5
64/64 [==============================] - 361s 6s/step - loss: 0.0697 - binary_accuracy: 0.9771 - accuracy: 0.9771
Epoch 5/5
64/64 [==============================] - 379s 6s/step - loss: 0.0589 - binary_accuracy: 0.9818 - accuracy: 0.9818


In [ ]:
model_training()

Epoch 1/10
64/64 [==============================] - 381s 6s/step - loss: 0.5585 - binary_accuracy: 0.7640 - accuracy: 0.7640
Epoch 2/10
64/64 [==============================] - 360s 6s/step - loss: 0.2406 - binary_accuracy: 0.8810 - accuracy: 0.8810
Epoch 3/10
64/64 [==============================] - 375s 6s/step - loss: 0.1899 - binary_accuracy: 0.9171 - accuracy: 0.9171
Epoch 4/10
64/64 [==============================] - 372s 6s/step - loss: 0.1589 - binary_accuracy: 0.9377 - accuracy: 0.9377
Epoch 5/10
64/64 [==============================] - 364s 6s/step - loss: 0.1369 - binary_accuracy: 0.9488 - accuracy: 0.9488
Epoch 6/10
64/64 [==============================] - 371s 6s/step - loss: 0.1241 - binary_accuracy: 0.9549 - accuracy: 0.9549
Epoch 7/10
64/64 [==============================] - 361s 6s/step - loss: 0.1110 - binary_accuracy: 0.9606 - accuracy: 0.9606
Epoch 8/10
64/64 [==============================] - 370s 6s/step - loss: 0.1004 - binary_accuracy: 0.9657 - accuracy: 0.9657


In [ ]:
# model.load_weights("/content/gdrive/My Drive/BBH_Classification.ckpt")


In [ ]:
# score = model.evaluate(val_x, val_y, verbose = 0) 

# print('Test loss:', score[0]) 
# print('Test accuracy:', score[1])

Test loss: 38.1667594909668
Test accuracy: 0.75


In [ ]:


# from keras.models import load_model
# model = load_model("/content/gdrive/MyDrive/BBH_Classification_Model_1.h5")